In [1]:
! pip install kaggle

In [2]:
# Upload kaggle.json file

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
# Importing Twitter Sentiment dataset

In [5]:
# API to fetch dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
# extracting the compressed dataset

from zipfile import ZipFile
dataset='sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print("The dataset is extracted")

The dataset is extracted


In [7]:
# Importing the Dependencies

In [8]:
import numpy as np
import pandas as pd
import re
# from nltk.corpus import stopwords
# from nltk.stem.porter import PorterStemmer
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.metrics import (
#     confusion_matrix,
#     ConfusionMatrixDisplay,
#     classification_report,
#     f1_score
# )
# import matplotlib.pyplot as plt

# import gensim.downloader as api
# import torch
# from transformers import BertTokenizer, BertForSequenceClassification



In [9]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/snehsoni/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [11]:
# Data Processing

In [12]:
# load the data from csv file to pandas dataframe
twitter_data=pd.read_csv('training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

In [13]:
twitter_data.shape

(1599999, 6)

In [14]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [15]:
# naming the columns and reading dataset again

column_names=['target','id','date','flag','user','text']
twitter_data=pd.read_csv('training.1600000.processed.noemoticon.csv',names=column_names,encoding='ISO-8859-1')

In [16]:
twitter_data.shape

(1600000, 6)

In [17]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [18]:
# counting the no of missing values in dataset

twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [19]:
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [20]:
# Convert the target '4' to '1'

In [21]:
twitter_data.replace({'target':{4:1}},inplace=True)

In [22]:
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [23]:
# 0 --> Negative Tweet
# 1 --> Positive Tweet

In [24]:
# Stemming

In [25]:
from nltk.stem.porter import PorterStemmer
port_stem= PorterStemmer()

In [26]:
def stemming(content):
    stemmed_content=re.sub('[^a-zA-Z]',' ',content)
    stemmed_content=stemmed_content.lower()
    stemmed_content=stemmed_content.split()
    stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content=' '.join(stemmed_content)

    return stemmed_content

In [ ]:
twitter_data['stemmed_content']=twitter_data['text'].apply(stemming)

In [ ]:
twitter_data.head()

In [ ]:
print(twitter_data['stemmed_content'])

In [ ]:
print(twitter_data['target'])

In [ ]:
# seperating data and label

X=twitter_data['stemmed_content'].values
Y=twitter_data['target'].values

In [ ]:
print(X)

In [ ]:
print(Y)

In [ ]:
# Splitting the data to training data and test data

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [ ]:
print(X.shape,X_train.shape,X_test.shape)

In [ ]:
print(X_train)

In [ ]:
print(X_test)

In [ ]:
# convert the actual data to numerical data

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer()

X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

In [ ]:
# * Store TF-IDF processed train/test for reuse
Xtr, Xte = X_train, X_test
ytr, yte = Y_train, Y_test


In [ ]:
print(X_train)

In [ ]:
print(X_test)

In [ ]:
#  Training the ML Model

In [ ]:
# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(max_iter=1000) 

In [ ]:
model.fit(X_train,Y_train)

In [ ]:
#* Logistic Regression model (explicit)
lr = LogisticRegression(max_iter=1000)
lr.fit(Xtr, ytr)

lr_pred_test = lr.predict(Xte)


In [ ]:
#* Store train/test data for later models (re-use in Naive Bayes, etc.)
Xtr, Xte = X_train, X_test
ytr, yte = Y_train, Y_test


In [ ]:
# *
from sklearn.naive_bayes import MultinomialNB

# Train Naive Bayes
nb = MultinomialNB(alpha=0.5)
nb.fit(Xtr, ytr)

# Predictions
nb_pred_test = nb.predict(Xte)


In [ ]:
# *
from sklearn.metrics import (
    confusion_matrix,
    ConfusionMatrixDisplay,
    classification_report,
    f1_score
)
import matplotlib.pyplot as plt
import numpy as np

def evaluate_model(name, y_true, y_pred, labels=None, label_names=None):
    print(f"=== {name} ===")
    print(f"Accuracy : {accuracy_score(y_true, y_pred):.4f}")
    print(f"F1 (macro)   : {f1_score(y_true, y_pred, average='macro'):.4f}")
    print(f"F1 (weighted): {f1_score(y_true, y_pred, average='weighted'):.4f}\n")

    print("Classification Report:")
    print(classification_report(y_true, y_pred, zero_division=0))

    cm = confusion_matrix(y_true, y_pred, labels=labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_names if label_names else labels)
    disp.plot(xticks_rotation=45, values_format='d')
    plt.title(f"{name} – Confusion Matrix (Counts)")
    plt.show()


In [ ]:
# labels = np.unique(ytr)
# label_names = [str(l) for l in labels]  # e.g., ["negative", "positive"]

# # Logistic Regression evaluation
# evaluate_model("Logistic Regression", yte, prediction, labels=labels, label_names=label_names)

# # Naive Bayes evaluation
# evaluate_model("Naive Bayes", yte, nb_pred_test, labels=labels, label_names=label_names)


# 1️⃣ Define the evaluation function
from sklearn.metrics import (
    confusion_matrix,
    ConfusionMatrixDisplay,
    classification_report,
    f1_score,
    accuracy_score
)
import matplotlib.pyplot as plt
import numpy as np

def evaluate_model(name, y_true, y_pred, labels=None, label_names=None):
    print(f"=== {name} ===")
    print(f"Accuracy : {accuracy_score(y_true, y_pred):.4f}")
    print(f"F1 (macro)   : {f1_score(y_true, y_pred, average='macro'):.4f}")
    print(f"F1 (weighted): {f1_score(y_true, y_pred, average='weighted'):.4f}\n")

    print("Classification Report:")
    print(classification_report(y_true, y_pred, zero_division=0))

    cm = confusion_matrix(y_true, y_pred, labels=labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_names if label_names else labels)
    disp.plot(xticks_rotation=45, values_format='d')
    plt.title(f"{name} – Confusion Matrix (Counts)")
    plt.show()


# 2️⃣ Train models
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

logreg_model = LogisticRegression(max_iter=1000)
logreg_model.fit(Xtr, ytr)

nb_model = MultinomialNB()
nb_model.fit(Xtr, ytr)

# 3️⃣ Make predictions
logreg_pred = logreg_model.predict(Xte)
nb_pred = nb_model.predict(Xte)

# 4️⃣ Prepare labels
labels = np.unique(yte)
label_names = [str(l) for l in labels]

# 5️⃣ Evaluate
evaluate_model("Logistic Regression", yte, logreg_pred, labels=labels, label_names=label_names)
evaluate_model("Naive Bayes", yte, nb_pred, labels=labels, label_names=label_names)


In [ ]:
# Model Evaluation

In [ ]:
# Accuracy score

In [ ]:
# accuracy score on the training data
from sklearn.metrics import accuracy_score

X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(Y_train,X_train_prediction)

In [ ]:
print('Accuracy score on the training data:',training_data_accuracy)

In [ ]:
# accuracy score on the test data
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(Y_test,X_test_prediction)

In [ ]:
print('Accuracy score on the test data:',test_data_accuracy)

In [ ]:
# Saving the trained model

In [ ]:
import pickle

In [ ]:
filename = "trained_model.pkl"
pickle.dump(model, open(filename, "wb"))

In [ ]:
# ✅ Added: save vectorizer also
with open("tfidf_vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)


In [ ]:
# Using the saved model for future predictions

In [ ]:
#  loading the saved model
loaded_model = pickle.load(open('trained_model.pkl', "rb"))

In [ ]:
# ✅ Added: load vectorizer also
with open("tfidf_vectorizer.pkl", "rb") as f:
    loaded_vectorizer = pickle.load(f)


In [ ]:
X_new=X_test[200]
print(Y_test[200])

prediction = loaded_model.predict(X_new)
print(prediction)

if(prediction[0]==0):
    print('Negative Tweet')
else:
    print('Positive Tweet')

In [ ]:
X_new=X_test[3]
print(Y_test[3])

prediction = loaded_model.predict(X_new)
print(prediction)

if(prediction[0]==0):
    print('Negative Tweet')
else:
    print('Positive Tweet')